In [1]:
import sys
import numpy as np
import json
import transformers
from transformers import CamembertForMaskedLM
from transformers import CamembertTokenizer as token
from transformers import CamembertForSequenceClassification
from transformers import AutoModel, AutoTokenizer, BertTokenizer
from transformers import CamembertConfig,CamembertTokenizer,CamembertModel
import tokenizers
import pandas as pd
sys.path.append('../')
import lib
import os
import torch

# 1 Load Article dataset

In [2]:
df=pd.DataFrame()
directory = os.path.join("../","data/data/")
for root,dirs,files in os.walk(directory):
    for file in files :
        if file.endswith(".json") and file.startswith("articles"):
            with open(root+file, 'r') as reader:
                data = json.load(reader)
            df_ = pd.DataFrame(columns=['id', 'title','text','Date','country','source', 'tweet'], 
                 data=data['articles'])
            df=pd.concat([df,df_],axis=0,ignore_index=True)

In [3]:
#remove duplicates if some news scraped twice
print(df.shape)
df=df.loc[df[['title', 'text', 'Date', 'country', 'source', 'tweet']].drop_duplicates().index]
print(df.shape)


(37014, 7)
(30855, 7)


In [4]:
df["text"]=df["text"].astype(str)

In [5]:
#884 rows with no text --> that's why we will create a column called tweet + text
print(len(df[df["text"].str.len()<1]))
#This means that there is no article so we will just add the tweet  to the text column (there is no link normally)
indexes=df[df["text"].str.len()<1].index
df.loc[indexes,"text"]=df.loc[indexes,"tweet"]

744


In [6]:
#verify it changed
len(df[df["text"].str.len()<1])

0

In [7]:
df["country"].unique()

array(['Maroc', 'Algeria', 'Guinee', 'Togo', 'Ile Maurice', 'Ivory Coast',
       'Cameroun', 'Niger', 'RDC', 'Burundi', 'Mali', 'Benin', 'Burkina',
       'Tunisia', 'Comores', 'Madagascar', 'Congo', 'Senegal', 'Gabon',
       'Centrafrique'], dtype=object)

In [8]:
id_=np.random.randint(df.shape[0])
print(id_)
df["text"].iloc[id_]

22805


'Le président afghan Ashraf Ghani a approuvé la libération de centaines de prisonniers talibans dans le cadre des efforts visant à ouvrir la voie au début des pourparlers avec les militants.\n\nSediq Sediqqi, porte-parole du palais présidentiel afghan, a annoncé mardi dans un tweet anglais que Ghani avait signé un décret pour libérer 1 500 prisonniers talibans cette semaine. « Le président Ghani a signé le décret qui faciliterait la libération des prisonniers talibans conformément à un cadre accepté pour le début des négociations entre les talibans et le gouvernement afghan », a déclaré le porte-parole dans son tweet. Sediqqi a annoncé dans des tweets ultérieurs en persan que l’ensemble des 1 500 prisonniers seraient libérés dans les 15 jours à compter de samedi, « avec 100 prisonniers sortant chaque jour des prisons afghanes », rapporte Press TV.\n\nLe responsable a également souligné qu’une fois les pourparlers directs entre les Taliban et Kaboul commencés, le gouvernement afghan lib

In [9]:
# check errors for future tokenization 
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
tokens = tokenizer.tokenize("coronavirus Covid-19 +")
print("Tokens: {}".format(tokens))

#Probleme avec noms de famille africain Alassane Ouattara (president Cote D'ivoire)/Joseph Dion Ngute (President Cameroun)
tokens = tokenizer.tokenize("Donald Trump Obama  Nicolas Sarkozy ||| Alassane Ouattara Arnauld || Joseph Dion Ngute")
print("Tokens: {}".format(tokens))

#Should we replace presidents names by President ??

Tokens: ['▁cor', 'ona', 'virus', '▁Co', 'vid', '-19', '▁+']
Tokens: ['▁Donald', '▁Trump', '▁Obama', '▁Nicolas', '▁Sarkozy', '▁|', '|', '|', '▁Al', 'assan', 'e', '▁Ouattara', '▁Arn', 'au', 'ld', '▁|', '|', '▁Joseph', '▁Dion', '▁N', 'gu', 'te']


In [10]:
for column in ["tweet","text"]:
    df[column] = df[column].replace('\n', ' ',regex=True)
    df[column] = df[column].replace("  ", " ")
    df[column] = df[column].replace('[^\w\s]','')
    df[column] = df[column].replace('Covid-19','coronavirus')
    df[column] = df[column].replace('covid-19','coronavirus')
    df[column] = df[column].replace('covid 19','coronavirus')
    df[column] = df[column].replace('Covid 19','coronavirus')
    df[column] = df[column].replace('➡️',' ')

In [11]:
id_=np.random.randint(df.shape[0])
print(id_)
df["text"].iloc[id_]

28031


'© Koaci.com - jeudi 26 mars 2020 - 18:18  Un trottro à Accra (ph)  Les conducteurs des minibus de transports publics au Ghana communément appelés « trottos » pourront aider à la lutte contre la propagation du coronavirus dans le pays. Quant à la situation sur le coronavirus, il est enregistré ce jour 64 nouveaux cas, portant le total à 132.  Dans la perspective de réduire le nombre de passagers dans les trottros, le Syndicat des transporteurs privés au Ghana (GPRTU) à Accra propose aux conducteurs et à leurs apprentis « mates » de réduire le nombre de passagers à bord de leurs engins.  Pour la phase concrète de cette recommandation, le GPRTU souhaite que les trottros qui prennent par exemple 3 passagers par rangée réduisent ce chargement à 2. En ce qui concerne ceux qui prennent 4 passagers par rangée, qu’ils le réduisent à 3 par rangée.  En attendant que le gouvernement se penche sur cette recommandation et apporte son concours le GPRTU a émis l’idée que les passagers payent un peu p

# 2 Load Data for Deaths and Reported Cases per country

In [12]:
import datetime 
!pwd

/Users/williampalmer/Desktop/AM221/risk_analysis_covid/notebooks


In [13]:
df_deaths=pd.read_csv("../data/time_series_covid19_deaths_global.csv")
df_cases=pd.read_csv("../data/time_series_covid19_confirmed_global.csv")

columns_deaths=df_deaths.columns.values
columns_cases=df_cases.columns.values
columns_deaths==columns_cases

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [14]:
columns_deaths

array(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20',
       '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20',
       '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20',
       '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20',
       '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20',
       '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20',
       '3/17/20', '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20',
       '3/23/20', '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20',
       '3/29/20', '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20',
       '4/4/20', '4/5/20', '4/6/20', '4/7/20', '4/8/20', '4/9/20',
       '4

In [15]:
columns_to_select=['Country/Region']+columns_deaths[4:].tolist()


In [16]:
df_cases.set_index("Country/Region").stack().reset_index()

,Country/Region,level_1,0
0,Afghanistan,Lat,33
1,Afghanistan,Long,65
2,Afghanistan,1/22/20,0
3,Afghanistan,1/23/20,0
4,Afghanistan,1/24/20,0
...,...,...,...
24629,Yemen,4/17/20,1
24630,Yemen,4/18/20,1
24631,Yemen,4/19/20,1
24632,Yemen,4/20/20,1


In [17]:
df_deaths=df_deaths.groupby(["Country/Region"]).sum().reset_index()
df_cases=df_cases.groupby(["Country/Region"]).sum().reset_index() #to sum when data from different province
df_deaths.set_index("Country/Region").stack().reset_index()

,Country/Region,level_1,0
0,Afghanistan,Lat,33.0
1,Afghanistan,Long,65.0
2,Afghanistan,1/22/20,0.0
3,Afghanistan,1/23/20,0.0
4,Afghanistan,1/24/20,0.0
...,...,...,...
17200,Zimbabwe,4/17/20,3.0
17201,Zimbabwe,4/18/20,3.0
17202,Zimbabwe,4/19/20,3.0
17203,Zimbabwe,4/20/20,3.0


In [18]:
# In order to concat
df_deaths=df_deaths[columns_to_select].set_index("Country/Region").stack().reset_index()
df_cases=df_cases[columns_to_select].set_index("Country/Region").stack().reset_index()

In [19]:
df_deaths=df_deaths.rename(columns={0:'Number_Deaths','level_1':'Date_nbr'})
df_cases=df_cases.rename(columns={0:'Number_Cases','level_1':'Date_nbr'})
df_cases

,Country/Region,Date_nbr,Number_Cases
0,Afghanistan,1/22/20,0
1,Afghanistan,1/23/20,0
2,Afghanistan,1/24/20,0
3,Afghanistan,1/25/20,0
4,Afghanistan,1/26/20,0
...,...,...,...
16830,Zimbabwe,4/17/20,24
16831,Zimbabwe,4/18/20,25
16832,Zimbabwe,4/19/20,25
16833,Zimbabwe,4/20/20,25


### Convert to date object

In [20]:
df_cases["Date_nbr"]=df_cases.Date_nbr.apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%y').date())
df_deaths["Date_nbr"]=df_deaths.Date_nbr.apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%y').date())


In [21]:
df["Date"]=df.Date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S').date())


In [22]:
df_deaths_and_cases=df_cases.merge(df_deaths,on=["Country/Region","Date_nbr"])
df_deaths_and_cases

,Country/Region,Date_nbr,Number_Cases,Number_Deaths
0,Afghanistan,2020-01-22,0,0
1,Afghanistan,2020-01-23,0,0
2,Afghanistan,2020-01-24,0,0
3,Afghanistan,2020-01-25,0,0
4,Afghanistan,2020-01-26,0,0
...,...,...,...,...
16830,Zimbabwe,2020-04-17,24,3
16831,Zimbabwe,2020-04-18,25,3
16832,Zimbabwe,2020-04-19,25,3
16833,Zimbabwe,2020-04-20,25,3


### Add the last 7 days type column

In [23]:
#Want to sub the 7th row before and because we have all the exact same number of rowas per country
#verify
df_deaths_and_cases["Number_Cases_Last_7_days"],df_deaths_and_cases["Number_Deaths_Last_7_days"]=np.nan,np.nan
for country in df_deaths_and_cases["Country/Region"].unique():
    df_c=df_deaths_and_cases[df_deaths_and_cases["Country/Region"]==country]
    indexes_=df_c.index
    to_substract_cases=df_c["Number_Cases"].loc[indexes_[:-7]]
    to_substract_cases.index=indexes_[7:]
    to_substract_deaths=df_c["Number_Cases"].loc[indexes_[:-7]]
    to_substract_deaths.index=indexes_[7:]
    df_deaths_and_cases.loc[indexes_[7:],["Number_Cases_Last_7_days"]]=df_c["Number_Cases"].loc[indexes_[7:]]-to_substract_cases
    df_deaths_and_cases.loc[indexes_[7:],["Number_Deaths_Last_7_days"]]=df_c["Number_Deaths"].loc[indexes_[7:]]-to_substract_deaths    
    
    

In [24]:
# add population number
df_population=pd.read_csv("../data/Population_countries.csv")
df_population=df_population[["Country Name",'2018']].rename(columns={"Country Name":"Country/Region","2018":'Population'})
df_deaths_and_cases=df_deaths_and_cases.merge(df_population,on=["Country/Region"])
df_deaths_and_cases

,Country/Region,Date_nbr,Number_Cases,Number_Deaths,Number_Cases_Last_7_days,Number_Deaths_Last_7_days,Population
0,Afghanistan,2020-01-22,0,0,NaN,NaN,37172386.0
1,Afghanistan,2020-01-23,0,0,NaN,NaN,37172386.0
2,Afghanistan,2020-01-24,0,0,NaN,NaN,37172386.0
3,Afghanistan,2020-01-25,0,0,NaN,NaN,37172386.0
4,Afghanistan,2020-01-26,0,0,NaN,NaN,37172386.0
...,...,...,...,...,...,...,...
14646,Zimbabwe,2020-04-17,24,3,11.0,-10.0,14439018.0
14647,Zimbabwe,2020-04-18,25,3,11.0,-11.0,14439018.0
14648,Zimbabwe,2020-04-19,25,3,11.0,-11.0,14439018.0
14649,Zimbabwe,2020-04-20,25,3,8.0,-14.0,14439018.0


In [25]:
#If we want to do the progression compared to other countries
#dist =  df_deaths_and_cases.groupby('Date_nbr').transform('max')
#df_all_test.join(dist.sub(df_all_test.drop('Date_nbr', axis=1)).div(dist).add_suffix('_%'))



# 3 Concat thinking about days before

In [26]:
df["Date_day_before"]=df.Date.apply(lambda x: x-- datetime.timedelta(days=-1))
df.head()

,id,title,text,Date,country,source,tweet,Date_day_before
0,Maroc_Leconomiste__0,Violence dans les stades: Arrêtons de tourner ...,"Aujourd’hui, le football maro­cain a le triste...",2020-02-25,Maroc,Leconomiste_,"Tribune - ""Violence dans les stades: Arrêtons ...",2020-02-24
1,Maroc_Leconomiste__1,Peu d'appétit pour les chèques repas,Le relèvement du plafond de déductibilité fisc...,2020-02-25,Maroc,Leconomiste_,Peu d'appétit pour les chèques repas http://ow...,2020-02-24
2,Maroc_Leconomiste__2,Résultats annuels: CIH Bank surfe sur une dyna...,Il semble que la relation inverse entre les ré...,2020-02-25,Maroc,Leconomiste_,Résultats annuels: CIH Bank surfe sur une dyna...,2020-02-24
3,Maroc_Leconomiste__3,Financement participatif: L’écosystème intègre...,Les actifs de la finance islamique mondiale po...,2020-02-25,Maroc,Leconomiste_,Financement participatif: L’écosystème intègre...,2020-02-24
4,Maroc_Leconomiste__4,"FMI: «Pour être riche, il faut investir dans l...",Les argentiers du monde entier se réuniront en...,2020-02-25,Maroc,Leconomiste_,"#FMI: «Pour être riche, il faut investir dans ...",2020-02-24


In [27]:
print(sorted(df.country.unique().tolist()))

['Algeria', 'Benin', 'Burkina', 'Burundi', 'Cameroun', 'Centrafrique', 'Comores', 'Congo', 'Gabon', 'Guinee', 'Ile Maurice', 'Ivory Coast', 'Madagascar', 'Mali', 'Maroc', 'Niger', 'RDC', 'Senegal', 'Togo', 'Tunisia']


In [28]:
df_all=df.merge(df_deaths_and_cases,how='inner',left_on=['country','Date_day_before'],right_on=["Country/Region","Date_nbr"])
print(sorted(df_all.country.unique().tolist()))
#some missing need to change country name

['Benin', 'Burundi', 'Gabon', 'Madagascar', 'Mali', 'Niger', 'Senegal', 'Togo', 'Tunisia']


In [29]:
print(df_deaths["Country/Region"].unique())

['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'Brunei' 'Bulgaria' 'Burkina Faso' 'Burma' 'Burundi'
 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada' 'Central African Republic'
 'Chad' 'Chile' 'China' 'Colombia' 'Congo (Brazzaville)'
 'Congo (Kinshasa)' 'Costa Rica' "Cote d'Ivoire" 'Croatia' 'Cuba' 'Cyprus'
 'Czechia' 'Denmark' 'Diamond Princess' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 'Estonia' 'Eswatini' 'Ethiopia' 'Fiji' 'Finland' 'France'
 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana' 'Greece' 'Grenada'
 'Guatemala' 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti' 'Holy See'
 'Honduras' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran' 'Iraq'
 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Japan' 'Jordan' 'Kaz

In [30]:
df.country.unique()

array(['Maroc', 'Algeria', 'Guinee', 'Togo', 'Ile Maurice', 'Ivory Coast',
       'Cameroun', 'Niger', 'RDC', 'Burundi', 'Mali', 'Benin', 'Burkina',
       'Tunisia', 'Comores', 'Madagascar', 'Congo', 'Senegal', 'Gabon',
       'Centrafrique'], dtype=object)

In [31]:
df_renamed=df
df_renamed["country"]=df_renamed["country"].str.replace('Burkina','Burkina Faso')
df_renamed["country"]=df_renamed["country"].str.replace('Cameroun','Cameroon')
df_renamed["country"]=df_renamed["country"].str.replace('Centrafrique','Central African Republic')
#Pas de comores ????
df_renamed["country"]=df_renamed["country"].str.replace('Congo','Congo (Brazzaville)')
df_renamed["country"]=df_renamed["country"].str.replace('Guinee','Guinea')
df_renamed["country"]=df_renamed["country"].str.replace('Ile Maurice','Mauritius')
df_renamed["country"]=df_renamed["country"].str.replace('Ivory Coast',"Cote d'Ivoire")
df_renamed["country"]=df_renamed["country"].str.replace('Maroc','Morocco')
df_renamed["country"]=df_renamed["country"].str.replace('RDC','Congo (Kinshasa)')

df_renamed.country.unique()

array(['Morocco', 'Algeria', 'Guinea', 'Togo', 'Mauritius',
       "Cote d'Ivoire", 'Cameroon', 'Niger', 'Congo (Kinshasa)',
       'Burundi', 'Mali', 'Benin', 'Burkina Faso', 'Tunisia', 'Comores',
       'Madagascar', 'Congo (Brazzaville)', 'Senegal', 'Gabon',
       'Central African Republic'], dtype=object)

In [32]:
# try again
df_all=df_renamed.merge(df_deaths_and_cases,how='inner',left_on=['country','Date_day_before'],right_on=["Country/Region","Date_nbr"])
print(sorted(df_all.country.unique().tolist()))


['Benin', 'Burkina Faso', 'Burundi', 'Cameroon', 'Central African Republic', 'Congo (Brazzaville)', 'Congo (Kinshasa)', "Cote d'Ivoire", 'Gabon', 'Guinea', 'Madagascar', 'Mali', 'Mauritius', 'Morocco', 'Niger', 'Senegal', 'Togo', 'Tunisia']


In [33]:
df_all.head()


,id,title,text,Date,country,source,tweet,Date_day_before,Country/Region,Date_nbr,Number_Cases,Number_Deaths,Number_Cases_Last_7_days,Number_Deaths_Last_7_days,Population
0,Maroc_Leconomiste__0,Violence dans les stades: Arrêtons de tourner ...,"Aujourd’hui, le football maro­cain a le triste...",2020-02-25,Morocco,Leconomiste_,"Tribune - ""Violence dans les stades: Arrêtons ...",2020-02-24,Morocco,2020-02-24,0,0,0.0,0.0,36029138.0
1,Maroc_Leconomiste__1,Peu d'appétit pour les chèques repas,Le relèvement du plafond de déductibilité fisc...,2020-02-25,Morocco,Leconomiste_,Peu d'appétit pour les chèques repas http://ow...,2020-02-24,Morocco,2020-02-24,0,0,0.0,0.0,36029138.0
2,Maroc_Leconomiste__2,Résultats annuels: CIH Bank surfe sur une dyna...,Il semble que la relation inverse entre les ré...,2020-02-25,Morocco,Leconomiste_,Résultats annuels: CIH Bank surfe sur une dyna...,2020-02-24,Morocco,2020-02-24,0,0,0.0,0.0,36029138.0
3,Maroc_Leconomiste__3,Financement participatif: L’écosystème intègre...,Les actifs de la finance islamique mondiale po...,2020-02-25,Morocco,Leconomiste_,Financement participatif: L’écosystème intègre...,2020-02-24,Morocco,2020-02-24,0,0,0.0,0.0,36029138.0
4,Maroc_Leconomiste__4,"FMI: «Pour être riche, il faut investir dans l...",Les argentiers du monde entier se réuniront en...,2020-02-25,Morocco,Leconomiste_,"#FMI: «Pour être riche, il faut investir dans ...",2020-02-24,Morocco,2020-02-24,0,0,0.0,0.0,36029138.0


In [34]:
df_all=df_all.rename(columns={'Date':'Date_published_article'})

In [35]:
df_all[['id', 'title', 'text', 'Date_published_article','Date_nbr', 'country', 'source', 'tweet',
       'Date_day_before', 'Number_Cases',
       'Number_Deaths','Number_Cases_Last_7_days',
       'Number_Deaths_Last_7_days','Population']]

,id,title,text,Date_published_article,Date_nbr,country,source,tweet,Date_day_before,Number_Cases,Number_Deaths,Number_Cases_Last_7_days,Number_Deaths_Last_7_days,Population
0,Maroc_Leconomiste__0,Violence dans les stades: Arrêtons de tourner ...,"Aujourd’hui, le football maro­cain a le triste...",2020-02-25,2020-02-24,Morocco,Leconomiste_,"Tribune - ""Violence dans les stades: Arrêtons ...",2020-02-24,0,0,0.0,0.0,36029138.0
1,Maroc_Leconomiste__1,Peu d'appétit pour les chèques repas,Le relèvement du plafond de déductibilité fisc...,2020-02-25,2020-02-24,Morocco,Leconomiste_,Peu d'appétit pour les chèques repas http://ow...,2020-02-24,0,0,0.0,0.0,36029138.0
2,Maroc_Leconomiste__2,Résultats annuels: CIH Bank surfe sur une dyna...,Il semble que la relation inverse entre les ré...,2020-02-25,2020-02-24,Morocco,Leconomiste_,Résultats annuels: CIH Bank surfe sur une dyna...,2020-02-24,0,0,0.0,0.0,36029138.0
3,Maroc_Leconomiste__3,Financement participatif: L’écosystème intègre...,Les actifs de la finance islamique mondiale po...,2020-02-25,2020-02-24,Morocco,Leconomiste_,Financement participatif: L’écosystème intègre...,2020-02-24,0,0,0.0,0.0,36029138.0
4,Maroc_Leconomiste__4,"FMI: «Pour être riche, il faut investir dans l...",Les argentiers du monde entier se réuniront en...,2020-02-25,2020-02-24,Morocco,Leconomiste_,"#FMI: «Pour être riche, il faut investir dans ...",2020-02-24,0,0,0.0,0.0,36029138.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24697,Gabon_InfosGabon_516,Le sens du Devoir Civique : Discours à la Nati...,"Libreville, Jeudi 19 Mars 2020 (Infos Gabon) –...",2020-03-19,2020-03-18,Gabon,InfosGabon,Gabon / Le sens du Devoir Civique : Discours à...,2020-03-18,1,0,1.0,0.0,2119275.0
24698,Gabon_InfosGabon_517,Coronavirus : La SEEG dans le combat,"Libreville, Jeudi 19 Mars 2020 (Infos Gabon) –...",2020-03-19,2020-03-18,Gabon,InfosGabon,Coronavirus : La SEEG dans le combat. Cliquez ...,2020-03-18,1,0,1.0,0.0,2119275.0
24699,Gabon_InfosGabon_518,Plafonnement des prix : La DGCC veille au grain,"Libreville, Jeudi 19 Mars 2020 (Infos Gabon) –...",2020-03-19,2020-03-18,Gabon,InfosGabon,Gabon / Plafonnement des prix : La DGCC veille...,2020-03-18,1,0,1.0,0.0,2119275.0
24700,Gabon_InfosGabon_519,Lutte contre le Coronavirus au Gabon : L’espoi...,"Libreville, Jeudi 19 Mars 2020 (Infos Gabon) –...",2020-03-19,2020-03-18,Gabon,InfosGabon,Lutte contre le Coronavirus au Gabon : L’espoi...,2020-03-18,1,0,1.0,0.0,2119275.0


In [36]:
### Add percentage for the day amon African countries

# 4 Save

In [37]:

df_all.to_csv("../data"+"/Data_cleaned.csv",index=False)

In [38]:
# Store the model we want to use
MODEL_NAME = "camembert-base"

# 2 POSSIBILITIES TO LOAD CAMEMBERT

#1st
config = CamembertConfig.from_pretrained(MODEL_NAME)    # Download configuration from S3 and cache.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_config(config) 


#2nd
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
camembert = CamembertModel.from_pretrained("camembert-base")

camembert.eval() 

CamembertModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32005, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, in

In [39]:
# Tokens comes from a process that splits the input into sub-entities with interesting linguistic properties. 
tokens = tokenizer.tokenize("ceci est un exemple !")
print("Tokens: {}".format(tokens))

# This is not sufficient for the model, as it requires integers as input, 
# not a problem, let's convert tokens to ids.
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Tokens id: {}".format(tokens_ids))

# Add the required special tokens
tokens_ids = tokenizer.build_inputs_with_special_tokens(tokens_ids)

# We need to convert to a Deep Learning framework specific format, let's use PyTorch for now.
tokens_pt = torch.tensor([tokens_ids])
print("Tokens PyTorch: {}".format(tokens_pt))

# Now we're ready to go through BERT with out input
outputs, pooled = model(tokens_pt)
print("Token wise output: {}, Pooled output: {}".format(outputs.shape, pooled.shape))

Tokens: ['▁ceci', '▁est', '▁un', '▁exemple', '▁!']
Tokens id: [2384, 30, 23, 411, 83]
Tokens PyTorch: tensor([[   5, 2384,   30,   23,  411,   83,    6]])
Token wise output: torch.Size([1, 7, 768]), Pooled output: torch.Size([1, 768])


In [40]:
tokens = tokenizer.tokenize('Covid-19')

In [41]:
directory

'../data/data/'